In [2]:
import cv2
import numpy as np
import pyaudio
import pygame
import time
import random

pygame 2.5.2 (SDL 2.28.3, Python 3.10.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


- cv2 : OpenCV, untuk pengolahan gambar/video, rendering game.
- numpy : Operasi numerik dan array.
- pyaudio : Input audio real-time dari mikrofon.
- pygame : Memutar musik latar.
- time : Fungsi waktu.
- random : Membuat rintangan platform/gap secara acak.

In [3]:
def display_message(frame, text, position, size=1, color=(255, 255, 255)):
    font = cv2.FONT_HERSHEY_SIMPLEX
    text_size = cv2.getTextSize(text, font, size, 2)[0]
    text_x = int(position[0] - text_size[0] // 2)
    text_y = int(position[1])
    cv2.putText(frame, text, (text_x, text_y), font, size, color, 2, cv2.LINE_AA)

`display_message` Digunakan menampilkan teks di posisi tertentu pada frame OpenCV dengan font dan warna yang bisa diatur :
- `frame`: Gambar atau frame video tempat teks akan ditampilkan (biasanya hasil dari `cv2.VideoCapture().read()`).
- `text`: String teks yang ingin ditampilkan.
- `position`: Tuple `(x, y)` sebagai posisi pusat teks (bukan kiri-atas) dalam piksel.
- `size`: Ukuran font teks (default: `1`).
- `color`: Warna teks dalam format BGR (default: `(255, 255, 255)` = putih).
- `cv2.FONT_HERSHEY_SIMPLEX`: Pilihan jenis font (biasa digunakan di OpenCV).
- `cv2.getTextSize(...)`: Mengembalikan ukuran (lebar, tinggi) teks saat ditampilkan.
- `2` adalah thickness (ketebalan garis) untuk estimasi ukuran.
- `[0]` mengambil ukuran teks saja, bukan baseline (garis bawah huruf seperti "g", "y").
- `text_x`: Menggeser posisi horizontal agar teks terpusat secara horizontal di `position[0]`.
- `text_y`: Tetap menempatkan teks pada `position[1]` (tidak dikoreksi untuk tinggi).
- `frame`: Gambar atau frame tempat menulis teks.
- `(text_x, text_y)`: Koordinat kiri-atas tempat teks mulai ditulis.
- `font`, `size`, `color`: Properti teks.
- `2`: Ketebalan garis teks.
- `cv2.LINE_AA`: Anti-aliasing untuk membuat teks lebih halus.

In [4]:
def show_game_over_screen(frame, frame_w, frame_h, score):
    frame[:] = 0
    font = cv2.FONT_HERSHEY_SIMPLEX

    cv2.putText(frame, "GAME OVER", (frame_w // 2 - 120, frame_h // 3), font, 2, (0, 0, 255), 3, cv2.LINE_AA)
    cv2.putText(frame, f"Score: {score}", (frame_w // 2 - 80, frame_h // 2), font, 1.5, (0, 255, 0), 3, cv2.LINE_AA)
    cv2.putText(frame, "Press SPACE to Restart", (frame_w // 2 - 150, int(frame_h // 1.5)), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "Press ESC or Q to Exit", (frame_w // 2 - 140, int(frame_h // 1.3)), font, 1, (0, 0, 255), 2, cv2.LINE_AA)

    try:
        with open("leaderboard.txt", "r") as f:
            scores = [line.strip() for line in f.readlines()]
        cv2.putText(frame, "Leaderboard (Top 5):", (frame_w // 2 - 150, int(frame_h // 1.15)), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
        for i, s in enumerate(scores):
            y_pos = int(frame_h // 1.1) + i * 30
            cv2.putText(frame, f"{i+1}. {s}", (frame_w // 2 - 100, y_pos), font, 0.8, (200, 200, 200), 2)
    except:
        pass

`show_game_over_screen` Menampilkan layar "GAME OVER" lengkap dengan skor, instruksi restart/exit, dan leaderboard (dibaca dari file `leaderboard.txt`) :
- `frame`: Frame gambar (biasanya frame video atau canvas OpenCV) yang akan diubah menjadi layar game over.
- `frame_w`: Lebar frame dalam piksel.
- `frame_h`: Tinggi frame dalam piksel.
- `score`: Skor akhir permainan yang akan ditampilkan.
- `cv2.LINE_AA`: untuk anti-aliasing agar teks halus.
- Membaca file `leaderboard.txt` yang berisi skor tertinggi (asumsi satu skor per baris).
- Menghilangkan karakter newline (`strip()`).
- Jika file `leaderboard.txt` tidak ada atau error saat membaca, program tidak menampilkan leaderboard tapi juga tidak error (langsung dilewati).

In [5]:
def generate_ground_block(frame_w, frame_h, last_blocks):
    recent = last_blocks[-3:] if len(last_blocks) >= 3 else last_blocks
    if sum(1 for b in recent if b['type'] == 'gap') >= 1:
        block_type = "platform"
    else:
        block_type = random.choices(["platform", "gap", "platform"], weights=[5, 1, 5], k=1)[0]

    block_width = random.randint(100, 140) if block_type == "platform" else random.randint(40, 60)
    block_height = 20
    gap = random.randint(50, 80)
    y = random.randint(int(frame_h * 0.7), int(frame_h * 0.8))

    if last_blocks:
        last_block = last_blocks[-1]
        x = last_block['x'] + last_block['w'] + gap
    else:
        x = frame_w + gap

    return {
        "x": int(x),
        "y": int(y),
        "w": int(block_width),
        "h": int(block_height),
        "type": block_type
    }

`generate_ground_block` Membuat blok platform baru atau gap secara acak, dengan logika untuk menghindari terlalu banyak gap berturut-turut :
- `frame_w`: Lebar frame atau layar game.
- `frame_h`: Tinggi frame atau layar game.
- `last_blocks`: List berisi blok-blok terakhir yang sudah dibuat, setiap elemen adalah dictionary yang berisi informasi blok (posisi, ukuran, tipe).
- `random.choices`: Menghasilkan list, ambil indeks `[0]` untuk mendapatkan string tipe blok.
- Jika tipe `"platform"`, lebar blok random antara 100 sampai 140 piksel.
- Jika tipe `"gap"`, lebar blok lebih kecil, antara 40 sampai 60 (mungkin ini untuk celah area kosong yang relatif pendek).
- Kembalikan dictionary yang mendeskripsikan blok baru dengan posisi `X`, `Y`, lebar `w`, tinggi `h`, dan tipe blok (`"platform"` atau `"gap"`).

In [6]:
def reset_game(state, frame_h, frame_w, block_image, char_scale):
    state.update({
        'jumping': True,
        'y_velocity': 0,
        'score': 0,
        'previous_pitch': 0,
        'previous_volume': 0,
        'game_over': False,
        'game_started': True,
        'ground_blocks': []
    })
    y = int(frame_h * 0.65)
    for i in range(4):
        state['ground_blocks'].append({
            "x": i * 160,
            "y": int(frame_h * 0.75),
            "w": 140,
            "h": 20,
            "type": "platform"
        })
    for i in range(4, 8):
        block = generate_ground_block(frame_w, frame_h, state['ground_blocks'])
        block["x"] = i * 160
        state['ground_blocks'].append(block)

    state['y'] = -int(frame_h * char_scale)
    state['x'] = frame_w // 3

`reset_game` Mereset state permainan dan membuat platform awal (8 blok total, 4 platform awal, sisanya acak) :
- `state`: dictionary yang menyimpan seluruh status dan variabel game saat ini (posisi karakter, skor, status game, dll).
- `frame_h`: tinggi frame/layar game.
- `frame_w`: lebar frame/layar game.
- `block_image`: gambar blok tanah/platform (meskipun dalam fungsi ini tidak dipakai secara langsung).
- `char_scale`: skala ukuran karakter, untuk menyesuaikan posisi vertikal karakter.
- `jumping` diset `True`, mungkin artinya karakter mulai dalam kondisi lompat.
- `y_velocity` = 0, kecepatan vertikal karakter direset.
- `score = 0`, skor direset saat mulai ulang game.
- `previous_pitch` & `previous_volume` = 0, kemungkinan ini variabel untuk kontrol suara/mikrofon di game.
- `game_over` diset `False`, artinya game belum berakhir.
- `game_started` diset `True`, game sudah mulai berjalan.
- `ground_blocks` direset jadi list kosong, siap diisi ulang blok-blok baru.
- Posisi vertikal blok di 75% dari tinggi frame (`y = frame_h * 0.75`).
- Gunakan fungsi `generate_ground_block` untuk membuat blok dengan variasi tipe (`platform` atau `gap`).
- Menambahkan blok-blok ini ke list `ground_blocks`.

In [7]:
pygame.mixer.init()
background_music = pygame.mixer.Sound("resources/background_music.mp3")
jump_effect = pygame.mixer.Sound("resources/jump.mp3")

dark_overlay = np.zeros((480, 640, 3), dtype=np.uint8)
cv2.rectangle(dark_overlay, (0, 0), (640, 480), (0, 0, 50), -1)

cap = cv2.VideoCapture(0)
ret, frame = cap.read()
frame_h, frame_w = frame.shape[:2]

p = pyaudio.PyAudio()
device_index = 1  # Bisa diubah sesuai device mic Anda

rate = 44100
chunk = 1024
stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=rate,
                input=True,
                input_device_index=device_index,
                frames_per_buffer=chunk)

character_img = cv2.imread("resources/piyik.png", cv2.IMREAD_UNCHANGED)
block_image = cv2.imread("resources/Block.jpg", cv2.IMREAD_UNCHANGED)

character_scale = 0.15
character = cv2.resize(character_img, (int(frame_w * character_scale), int(frame_w * character_scale)))
character_jump = character
block_image_resized = cv2.resize(block_image, (140, 20))
character_height, character_width = character.shape[:2]

score_anim = {
    'active': False,
    'scale': 1.5,
    'alpha': 1.0,
    'pos_y': 40,
    'timer': 20,
    'pulse_dir': 1,
    'color_phase': 0
}

state = {
    'speed': 5,
    'jumping': False,
    'y_velocity': 0,
    'gravity': 1,
    'score': 0,
    'highscore': 0,
    'volume_threshold': 600,
    'volume_max': 15000,
    'game_over': False,
    'game_started': False,
    'ground_blocks': [],
    'x': frame_w // 3,
    'y': int(frame_h * 0.65)
}

MIN_JUMP_VELOCITY = -15
MAX_JUMP_VELOCITY = -40

- `pygame.mixer.init()`: Menginisialisasi modul audio dari Pygame untuk memutar suara.
- `background_music`: Memuat file musik latar dari folder resources.
- `jump_effect`: Memuat efek suara lompat dari file MP3.
- Membuat array `dark_overlay` berukuran 480x640 piksel dengan 3 channel warna (BGR), diisi nol (hitam).
- `cv2.rectangle` menggambar persegi panjang penuh (`-1` untuk fill) warna biru gelap `(0, 0, 50)` di seluruh overlay.
- Membuka webcam (`cap = cv2.VideoCapture(0)`).
- Mendapatkan ukuran frame video: tinggi `(frame_h)` dan lebar `(frame_w)`.
- Membuat objek `PyAudio` untuk pengambilan audio.
- `device_index`: Mengacu ke mikrofon tertentu (nomor 1 di sini, bisa disesuaikan).
- `rate` = 44100 Hz, standar frekuensi sampling audio.
- `chunk` = 1024, ukuran buffer pengambilan data audio.
- `stream`: Membuka stream input audio dengan konfigurasi tersebut, siap membaca suara mikrofon.
- Memuat gambar karakter `(piyik.jpg)` dan blok `(Block.jpg)`, lalu resize sesuai ukuran frame.
- `IMREAD_UNCHANGED` menjaga agar channel alpha/transparansi tetap terbaca.
- Mengatur ukuran karakter berdasarkan skala relatif terhadap lebar frame (`character_scale` = 0.15).
- `character_jump` diset sama dengan `character` (mungkin untuk animasi lompat yang nantinya berbeda).
- `block_image_resized` diubah ukurannya menjadi 140x20 px (ukuran blok platform).
- Mendapatkan ukuran karakter (`character_height`, `character_width`) dari hasil resize.
- Inisialisasi `state` dictionary berisi berbagai status permainan seperti kecepatan, posisi karakter, skor, status game, dll.
- `MIN_JUMP_VELOCITY` dan `MAX_JUMP_VELOCITY`: konstanta kecepatan lompat minimal dan maksimal.

In [8]:
def apply_jump_effect(sprite):
    overlay = sprite.copy()
    if overlay.shape[2] == 4:
        overlay = cv2.cvtColor(overlay, cv2.COLOR_BGRA2BGR)
    blue_tint = np.full_like(overlay, (255, 100, 0))
    cv2.addWeighted(blue_tint, 0.4, overlay, 0.6, 0, overlay)
    return overlay

`apply_jump_effect` memberi efek warna pada karakter saat lompat :
- `overlay = sprite.copy()`: Membuat salinan gambar agar tidak mengubah gambar asli.
- `overlay.shape`: Menghasilkan tuple (tinggi, lebar, jumlah_channel).
- `cv2.cvtColor(overlay, cv2.COLOR_BGRA2BGR)`: Jika ada 4 channel (BGRA), kita konversi menjadi 3 channel (BGR) saja.
- `np.full_like` membuat array baru dengan ukuran dan tipe data sama seperti overlay.
- `cv2.addWeighted` untuk menggabungkan dua gambar dengan bobot transparansi tertentu.

In [9]:
def draw_shadow(frame, x, y, width, height):
    shadow_color = (50, 50, 50)
    shadow_pos = (int(x + width // 2), int(y + height - 5))
    shadow_size = (width // 2, 10)
    cv2.ellipse(frame, shadow_pos, shadow_size, 0, 0, 360, shadow_color, -1)

`draw_shadow` menggambar bayangan karakter di bawahnya :
- `frame`: gambar/frame OpenCV (biasanya ndarray NumPy) tempat bayangan akan digambar.
- `x`, `y`: koordinat titik awal (biasanya posisi objek/sprite).
- `width`, `height`: ukuran objek/sprite, untuk menentukan ukuran dan posisi bayangan relatif terhadap objek.
- `cv2.ellipse` menggambar elips di `frame`

In [10]:
def draw_game_over_screen(frame, score, highscore, frame_w, frame_h, character_img=None):
    overlay = np.zeros_like(frame)
    for i in range(frame_h):
        alpha = 0.7 * (1 - i / frame_h)
        cv2.line(overlay, (0, i), (frame_w, i), (0, 0, 0), 1)
        cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

    font = cv2.FONT_HERSHEY_DUPLEX
    base_scale = 3.0
    base_thickness = 5

    text = "GAME OVER"
    text_size = cv2.getTextSize(text, font, base_scale, base_thickness)[0]
    x = (frame_w - text_size[0]) // 2
    y = frame_h // 3

    for dx, dy in [(-2,0), (2,0), (0,-2), (0,2)]:
        cv2.putText(frame, text, (x + dx, y + dy), font, base_scale, (0,0,150), base_thickness + 3, cv2.LINE_AA)
    cv2.putText(frame, text, (x, y), font, base_scale, (0,0,255), base_thickness, cv2.LINE_AA)

    score_text = f"Score: {score}"
    score_size = cv2.getTextSize(score_text, font, 2, 4)[0]
    score_x = (frame_w - score_size[0]) // 2
    score_y = y + 100

    for dx, dy in [(-2,0), (2,0), (0,-2), (0,2)]:
        cv2.putText(frame, score_text, (score_x + dx, score_y + dy), font, 2, (0,100,0), 6, cv2.LINE_AA)
    cv2.putText(frame, score_text, (score_x, score_y), font, 2, (0,255,0), 4, cv2.LINE_AA)

    # Highscore di bawah score
    highscore_text = f"Highscore: {highscore}"
    highscore_size = cv2.getTextSize(highscore_text, font, 1.5, 3)[0]
    highscore_x = (frame_w - highscore_size[0]) // 2
    highscore_y = score_y + 70

    for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
        cv2.putText(frame, highscore_text, (highscore_x + dx, highscore_y + dy), font, 1.5, (0, 0, 100), 3, cv2.LINE_AA)
    cv2.putText(frame, highscore_text, (highscore_x, highscore_y), font, 1.5, (255, 215, 0), 3, cv2.LINE_AA)  # Gold color

    pulse = (np.sin(time.time() * 3) + 1) / 2

    restart_text = "Press SPACE to Restart"
    exit_text = "Press ESC or Q to Exit"

    restart_size = cv2.getTextSize(restart_text, font, 1.2, 2)[0]
    exit_size = cv2.getTextSize(exit_text, font, 1.2, 2)[0]

    restart_x = (frame_w - restart_size[0]) // 2
    restart_y = highscore_y + 70

    exit_x = (frame_w - exit_size[0]) // 2
    exit_y = restart_y + 60

    restart_color = (int(0), int(255 * pulse), 0)
    exit_color = (int(255 * pulse), 0, 0)

    cv2.putText(frame, restart_text, (restart_x, restart_y), font, 1.2, restart_color, 2, cv2.LINE_AA)
    cv2.putText(frame, exit_text, (exit_x, exit_y), font, 1.2, exit_color, 2, cv2.LINE_AA)

    if character_img is not None:
        ch_h, ch_w = character_img.shape[:2]
        scale = 0.2
        resized = cv2.resize(character_img, (int(ch_w * scale), int(ch_h * scale)))
        cx = (frame_w - resized.shape[1]) // 2
        cy = exit_y + 50
        if cy + resized.shape[0] < frame_h:
            roi = frame[cy:cy+resized.shape[0], cx:cx+resized.shape[1]]
            if resized.shape[2] == 4:
                alpha_mask = resized[:, :, 3] / 255.0
                for c in range(3):
                    roi[:, :, c] = roi[:, :, c] * (1 - alpha_mask) + resized[:, :, c] * alpha_mask
            else:
                roi[:] = resized

    return frame

`draw_game_over_screen` menggambar overlay transparan dan teks game over dengan animasi warna :
- `frame`: gambar (numpy array) yang akan digambar overlay dan teks game over.
- `score`: skor permainan saat ini (integer).
- `highscore`: skor tertinggi (integer).
- `frame_w`, `frame_h`: lebar dan tinggi frame.
- `character_img` (optional): gambar karakter (biasanya dengan alpha channel), yang akan ditampilkan di bagian bawah layar game over.
- Membuat sebuah `overlay` hitam berukuran sama dengan `frame`.
- `cv2.line` menggambar garis hitam horizontal per baris.
- `cv2.addWeighted` menggabungkan `overlay` dengan frame asli dengan alpha berbeda per baris, hasilnya frame terlihat gelap terutama di atas, jadi efek background jadi gelap bertahap dari atas.
- Menggunakan font `HERSHEY_DUPLEX` dan ukuran besar `(base_scale=3.0)`.

In [11]:
cv2.namedWindow("Loud Scream")

clicked = [False]

def on_mouse(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        clicked[0] = True

cv2.setMouseCallback("Loud Scream", on_mouse)

- Fungsi callback mouse `on_mouse` untuk mulai permainan saat klik mouse.
- Window OpenCV dibuat bernama `"Loud Scream"` dan callback mouse diset.

In [12]:
base_speed = 5
scaling_factor = 0.05

volume_buffer = []
jump_cooldown_ms = 500
last_jump_time = 0

- `base_speed` dan `scaling_factor` mengatur kecepatan dasar dan peningkatan kecepatan berdasarkan skor.
- `volume_buffer` untuk smoothing input suara.
- `jump_cooldown_ms` dan `last_jump_time` mengatur cooldown agar lompatan tidak terlalu cepat berturut-turut.

In [13]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    key = cv2.waitKey(1) & 0xFF

    if not state['game_started']:
        text = "Click or Press SPACE to Start"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1.0
        thickness = 2
        text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
        text_x = max(0, (frame_w - text_size[0]) // 2)
        text_y = max(30, frame_h // 3)

        cv2.putText(frame, text, (text_x, text_y), font, font_scale, (255, 255, 255), thickness, cv2.LINE_AA)
        cv2.putText(frame, f"Highscore: {state['highscore']}", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 215, 255), 2) 
        cv2.imshow("Loud Scream", frame)

        if key == ord(' ') or clicked[0]:
            reset_game(state, frame_h, frame_w, block_image_resized, character_scale)
            state['game_started'] = True
            background_music.play(-1)
            clicked[0] = False
        elif cv2.getWindowProperty("Loud Scream", cv2.WND_PROP_VISIBLE) < 1:
            break

        continue

    data = stream.read(chunk, exception_on_overflow=False)
    audio_data = np.frombuffer(data, dtype=np.int16)

    volume = np.mean(np.abs(audio_data))

    volume_buffer.append(volume)
    if len(volume_buffer) > 5:
        volume_buffer.pop(0)
    smooth_volume = sum(volume_buffer) / len(volume_buffer)

    current_time = cv2.getTickCount() / cv2.getTickFrequency() * 1000

    if not state['jumping'] and smooth_volume > state['volume_threshold']:
        if current_time - last_jump_time > jump_cooldown_ms:
            norm_vol = min((smooth_volume - state['volume_threshold']) / (state['volume_max'] - state['volume_threshold']), 1.0)
            norm_vol = norm_vol ** 2
            jump_velocity = MIN_JUMP_VELOCITY + norm_vol * (MAX_JUMP_VELOCITY - MIN_JUMP_VELOCITY)
            state['jumping'] = True
            state['y_velocity'] = jump_velocity
            last_jump_time = current_time
            jump_effect.play()

    state['speed'] = base_speed + (state['score'] * scaling_factor)

    next_y = state['y'] + state['y_velocity']
    character_bottom_next = next_y + character_height

    on_platform = False
    landed = False

    for block in state['ground_blocks']:
        if block["type"] == "gap":
            continue
        block_top = block["y"]
        block_left = block["x"]
        block_right = block["x"] + block["w"]

        character_left = state['x']
        character_right = state['x'] + character_width

        horizontal_collision = (character_right > block_left) and (character_left < block_right)

        if horizontal_collision and state['y'] + character_height <= block_top and character_bottom_next >= block_top:
            state['y'] = block_top - character_height
            state['jumping'] = False
            state['y_velocity'] = 0
            on_platform = True
            if not landed:
                state['score'] += 1
            landed = True
            break

    if not on_platform:
        state['y'] = next_y
        state['y_velocity'] += state['gravity']

    for block in state['ground_blocks']:
        block['x'] -= state['speed']

    if state['ground_blocks'] and state['ground_blocks'][-1]["x"] < frame_w:
        state['ground_blocks'].append(generate_ground_block(frame_w, frame_h, state['ground_blocks']))

    state['ground_blocks'] = [b for b in state['ground_blocks'] if b["x"] + b["w"] > 0]

    if state['y'] + character_height > frame_h:
        background_music.stop()
        if state['score'] > state['highscore']:
            state['highscore'] = state['score']
        state['game_over'] = True

    for block in state['ground_blocks']:
        if block["type"] == "platform":
            resized_block = cv2.resize(block_image_resized, (int(block["w"]), int(block["h"])))
            region = frame[int(block["y"]):int(block["y"]) + int(block["h"]),
                           int(block["x"]):int(block["x"]) + int(block["w"])]
            if region.shape[:2] == resized_block.shape[:2]:
                region[:] = resized_block

    draw_shadow(frame, int(state['x']), int(state['y']), character_width, character_height)

    sprite = character_jump if state['jumping'] else character

    if state['jumping']:
        sprite = apply_jump_effect(sprite)

    y1 = max(0, int(state['y']))
    y2 = min(frame_h, int(state['y'] + character_height))
    x1 = max(0, int(state['x']))
    x2 = min(frame_w, int(state['x'] + character_width))

    roi = frame[y1:y2, x1:x2]
    sprite_cropped = sprite[0:(y2 - y1), 0:(x2 - x1)]

    if sprite_cropped.shape[2] == 4:
        sprite_cropped = cv2.cvtColor(sprite_cropped, cv2.COLOR_BGRA2BGR)

    if roi.shape[:2] == sprite_cropped.shape[:2] and roi.shape[2] == sprite_cropped.shape[2]:
        gray = cv2.cvtColor(sprite_cropped, cv2.COLOR_BGR2GRAY)
        _, mask = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
        bg = cv2.bitwise_and(roi, roi, mask=cv2.bitwise_not(mask))
        fg = cv2.bitwise_and(sprite_cropped, sprite_cropped, mask=mask)
        frame[y1:y2, x1:x2] = cv2.add(bg, fg)

    if state['game_over']:
        frame = draw_game_over_screen(frame, state['score'], state['highscore'], frame_w, frame_h, character_img)
        cv2.imshow("Loud Scream", frame)

        while True:
            key = cv2.waitKey(0) & 0xFF
            if key == ord(' '):
                reset_game(state, frame_h, frame_w, block_image_resized, character_scale)
                background_music.play(-1)
                state['game_over'] = False
                break
            elif key == 27 or key == ord('q'):
                background_music.stop()
                cap.release()
                cv2.destroyAllWindows()
                exit(0)

    else:
        if score_anim['active']:
            overlay = frame.copy()
            text = f"Score: {state['score']}"
            font = cv2.FONT_HERSHEY_SIMPLEX
            text_x = 20
            text_y = int(score_anim['pos_y'])

            if score_anim['color_phase'] == 0:
                color = (0, 255, 0)
            elif score_anim['color_phase'] == 1:
                color = (0, 255, 255)
            else:
                color = (0, 165, 255)

            cv2.putText(overlay, text, (text_x, text_y), font, score_anim['scale'], color, 3)

            alpha = score_anim['alpha']
            cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

            score_anim['scale'] += 0.03 * score_anim['pulse_dir']
            if score_anim['scale'] > 2.0:
                score_anim['pulse_dir'] = -1
            elif score_anim['scale'] < 1.5:
                score_anim['pulse_dir'] = 1

            score_anim['pos_y'] -= 1.5
            score_anim['alpha'] -= 0.05

            score_anim['timer'] -= 1
            if score_anim['timer'] % 5 == 0:
                score_anim['color_phase'] = (score_anim['color_phase'] + 1) % 3

            if score_anim['timer'] <= 0 or score_anim['alpha'] <= 0:
                score_anim['active'] = False
        else:
            cv2.putText(frame, f"Score: {state['score']}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Highscore: {state['highscore']}", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 215, 255), 2)

        cv2.putText(frame, f"Volume: {smooth_volume:.2f}", (10, frame_h - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        cv2.imshow("Loud Scream", frame)

    if cv2.getWindowProperty("Loud Scream", cv2.WND_PROP_VISIBLE) < 1:
        break

    if key == 27 or key == ord('q'):
        background_music.stop()
        break

Loop Utama Game:
- Loop terus berjalan selama kamera terbuka dan window aktif.
- Menampilkan instruksi klik atau tekan space, highscore, dan menunggu input untuk mulai game.
- Membaca buffer audio dari mikrofon.
- Menghitung volume suara dan melakukan smoothing dari buffer.
- Jika suara melebihi threshold dan cooldown terpenuhi, set karakter lompat dengan kecepatan lompat yang dipetakan dari volume suara.
- Menghitung posisi karakter berikutnya berdasarkan kecepatan lompat dan gravitasi.
- Deteksi apakah karakter mendarat di platform.
- Jika tidak mendarat, karakter jatuh ke bawah (gravitasi bertambah).
- Blok tanah bergerak ke kiri dengan kecepatan game.
- Menambahkan blok baru di ujung kanan secara acak.
- Menghapus blok yang sudah di luar layar.
- Jika karakter jatuh melewati batas bawah layar, game over, musik dihentikan, dan highscore diupdate.
- Jika tekan ESC atau Q di mode main, hentikan musik dan keluar loop.

In [14]:
stream.stop_stream()
stream.close()
p.terminate()
cap.release()
cv2.destroyAllWindows()

Setelah loop selesai:
- Stop stream audio.
- Tutup PyAudio.
- Lepaskan kamera.
- Tutup window OpenCV.